In [1]:
%%spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2020,application_1572292571909_0502,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2016,application_1572292571909_0498,pyspark,idle,Link,Link,
2017,application_1572292571909_0499,pyspark,idle,Link,Link,
2018,application_1572292571909_0500,pyspark,idle,Link,Link,
2019,application_1572292571909_0501,pyspark,idle,Link,Link,
2020,application_1572292571909_0502,pyspark,idle,Link,Link,✔


In [3]:
import pyspark.sql.functions as f

In [4]:
StopWords = ["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would"]

In [5]:
import subprocess

cmd = 'hdfs dfs -ls /data/texts'.split() 
files = subprocess.check_output(cmd).strip().split('\n')
for path in files:
    print (path)

Found 2 items
-rw-r--r--   3 hdfs hdfs     496309 2018-10-11 11:04 /data/texts/Portrait.txt
-rw-r--r--   3 hdfs hdfs    1580890 2018-10-11 11:04 /data/texts/Ulysses.txt

In [8]:
raw_lines = spark.read.text('/data/texts')\
    .withColumn('file', f.input_file_name())
raw_lines.show()

+--------------------+--------------------+
|               value|                file|
+--------------------+--------------------+
|                    |hdfs://rapid-777....|
|The Project Guten...|hdfs://rapid-777....|
|                    |hdfs://rapid-777....|
|This eBook is for...|hdfs://rapid-777....|
|no restrictions w...|hdfs://rapid-777....|
|it under the term...|hdfs://rapid-777....|
|eBook or online a...|hdfs://rapid-777....|
|                    |hdfs://rapid-777....|
|                    |hdfs://rapid-777....|
|      Title: Ulysses|hdfs://rapid-777....|
|                    |hdfs://rapid-777....|
| Author: James Joyce|hdfs://rapid-777....|
|                    |hdfs://rapid-777....|
|Release Date: Aug...|hdfs://rapid-777....|
|Last Updated: Aug...|hdfs://rapid-777....|
|                    |hdfs://rapid-777....|
|   Language: English|hdfs://rapid-777....|
|                    |hdfs://rapid-777....|
|Character set enc...|hdfs://rapid-777....|
|                    |hdfs://rap

In [16]:
lines_by_book = raw_lines \
    .withColumn('book', f.regexp_extract('file', '^.*/(.*)\.txt$', 1))\
    .drop('file')
lines_by_book.show()

+--------------------+-------+
|               value|   book|
+--------------------+-------+
|                    |Ulysses|
|The Project Guten...|Ulysses|
|                    |Ulysses|
|This eBook is for...|Ulysses|
|no restrictions w...|Ulysses|
|it under the term...|Ulysses|
|eBook or online a...|Ulysses|
|                    |Ulysses|
|                    |Ulysses|
|      Title: Ulysses|Ulysses|
|                    |Ulysses|
| Author: James Joyce|Ulysses|
|                    |Ulysses|
|Release Date: Aug...|Ulysses|
|Last Updated: Aug...|Ulysses|
|                    |Ulysses|
|   Language: English|Ulysses|
|                    |Ulysses|
|Character set enc...|Ulysses|
|                    |Ulysses|
+--------------------+-------+
only showing top 20 rows

In [17]:
lines_by_book_1 = lines_by_book.withColumn('count', f.size(f.split(f.col('value'), ' ')))
lines_by_book_1 = lines_by_book_1.where(f.col('value')!= '')
lines_by_book_1.groupby('book').agg({'count':'sum'}).show()

+--------+----------+
|    book|sum(count)|
+--------+----------+
|Portrait|     88641|
| Ulysses|    271297|
+--------+----------+

In [34]:
lines_by_book_2 = lines_by_book.withColumn('word', f.explode(f.split(f.col('value'), ' '))).where(f.length('word')>0)\
.select('book', f.lower(f.trim(f.col('word')))).alias('word').show()

+-------+-----------------+
|   book|lower(trim(word))|
+-------+-----------------+
|Ulysses|              the|
|Ulysses|          project|
|Ulysses|        gutenberg|
|Ulysses|            ebook|
|Ulysses|               of|
|Ulysses|         ulysses,|
|Ulysses|               by|
|Ulysses|            james|
|Ulysses|            joyce|
|Ulysses|             this|
|Ulysses|            ebook|
|Ulysses|               is|
|Ulysses|              for|
|Ulysses|              the|
|Ulysses|              use|
|Ulysses|               of|
|Ulysses|           anyone|
|Ulysses|         anywhere|
|Ulysses|               at|
|Ulysses|               no|
+-------+-----------------+
only showing top 20 rows

In [47]:
lines_by_book_2 = lines_by_book\
    .select('book', f.explode(f.split('value', '[\W_]+')).alias('word'))\
    .where(f.length('word') > 0)\
    .select('book', f.trim(f.lower(f.col('word'))).alias('word'))\
    .filter(f.col('word').isin(StopWords) == False)

In [48]:
lines_by_book_2.show(20)

+-------+------------+
|   book|        word|
+-------+------------+
|Ulysses|     project|
|Ulysses|   gutenberg|
|Ulysses|       ebook|
|Ulysses|     ulysses|
|Ulysses|       james|
|Ulysses|       joyce|
|Ulysses|       ebook|
|Ulysses|         use|
|Ulysses|      anyone|
|Ulysses|    anywhere|
|Ulysses|        cost|
|Ulysses|      almost|
|Ulysses|restrictions|
|Ulysses|  whatsoever|
|Ulysses|         may|
|Ulysses|        copy|
|Ulysses|        give|
|Ulysses|        away|
|Ulysses|         use|
|Ulysses|       terms|
+-------+------------+
only showing top 20 rows

In [49]:
lines_by_book_3 = lines_by_book_2.groupby('word').count().orderBy('count',ascending=False).show(20)

+-------+-----+
|   word|count|
+-------+-----+
|   said| 1823|
|stephen| 1004|
|  bloom| 1001|
|    one|  965|
|   like|  951|
|     mr|  855|
|    old|  603|
|    man|  563|
|    see|  521|
|   eyes|  502|
|   time|  498|
|   says|  494|
| father|  480|
|   back|  479|
|    two|  470|
|    god|  470|
|    yes|  434|
|   know|  432|
| little|  420|
|   good|  412|
+-------+-----+
only showing top 20 rows

In [55]:
from pyspark.sql.window import Window

w = Window.partitionBy(lines_by_book_2['word'])
c_d = lines_by_book_2.select('book').distinct().count()

lines_by_book_2.groupby('book', 'word').agg(f.lit(c_d).alias('c_d'), f.count('*').over(w).alias('i_d'),\
        (f.log(f.lit(c_d)/f.count('*').over(w))).alias('idf')).orderBy('idf', ascending=False).show() 

+--------+-----------+---+---+------------------+
|    book|       word|c_d|i_d|               idf|
+--------+-----------+---+---+------------------+
| Ulysses|     borneo|  2|  1|0.6931471805599453|
| Ulysses|delectation|  2|  1|0.6931471805599453|
| Ulysses|  boulevard|  2|  1|0.6931471805599453|
| Ulysses|       1903|  2|  1|0.6931471805599453|
| Ulysses|       bros|  2|  1|0.6931471805599453|
| Ulysses|  capacious|  2|  1|0.6931471805599453|
|Portrait|  cardboard|  2|  1|0.6931471805599453|
| Ulysses|     akasic|  2|  1|0.6931471805599453|
|Portrait|   chaplets|  2|  1|0.6931471805599453|
| Ulysses|  assessors|  2|  1|0.6931471805599453|
|Portrait|   colleges|  2|  1|0.6931471805599453|
| Ulysses|       babe|  2|  1|0.6931471805599453|
| Ulysses|  comerford|  2|  1|0.6931471805599453|
| Ulysses| bassoonist|  2|  1|0.6931471805599453|
| Ulysses| comerfords|  2|  1|0.6931471805599453|
| Ulysses|    blooded|  2|  1|0.6931471805599453|
| Ulysses|      coral|  2|  1|0.6931471805599453|
